# Script to compute the kinematics (for the first time)

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font
from scipy.interpolate import interp1d

from stressgait_analysis.dataset import StressGaitDataset

from src.stressgait_analysis.gait_helper_functions import compute_leg_kinematics, compute_arm_kinematics

%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import compute_HS, resample
import numpy as np

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True, specify_bouts=True, specify_speed=True)

In [ ]:
data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints)
    n_cycle = 0
    #iterate over individual cycles:
    for i in range(len(hs_events)-1):

        cycle_start = hs_events[i]
        if cycle_start < 30:
            continue
        #print(cycle_start)
        cycle_end = hs_events[i+1]
        leg_kinematics = compute_leg_kinematics(keypoints[cycle_start:cycle_end])
        arm_kinematics = compute_arm_kinematics(keypoints[cycle_start:cycle_end])

        #resample to 101 (=0%-100% cycle)
        leg_kinematics = resample(leg_kinematics, n=101)
        arm_kinematics = resample(arm_kinematics, n=101)

        data.extend([
             (*trial.index.values[0], n_cycle, j, *leg_row, *arm_row)
                for j, (leg_row, arm_row) in enumerate(zip(leg_kinematics.values, arm_kinematics.values))
                ])
        n_cycle += 1

In [ ]:
kinematics_df = pd.DataFrame(data, columns=[*dataset.index.columns, "cycle_idx", "percentage_of_stride", *leg_kinematics.columns, *arm_kinematics.columns],).drop(columns = "ankle_plantarflexion")
kinematics_df.set_index([*dataset.index.columns, "cycle_idx", "percentage_of_stride"], inplace=True)

In [ ]:
kinematics_df.head()

## store the data

In [ ]:
filename = dataset.base_path.joinpath("kinematics/kinematics.csv")
kinematics_df.to_csv(filename)


# Great! Kinematics can now be loaded directly from the dataset!
